In [ ]:
import os

%load_ext dotenv
%dotenv

FEATURE_COLUMNS = ['player_rating_home_player_1', 'player_rating_home_player_2', 'player_rating_home_player_3',
                   'player_rating_home_player_4', 'player_rating_home_player_5',
                   'player_rating_home_player_6', 'player_rating_home_player_7', 'player_rating_home_player_8',
                   'player_rating_home_player_9', 'player_rating_home_player_10',
                   'player_rating_home_player_11', 'player_rating_away_player_1', 'player_rating_away_player_2',
                   'player_rating_away_player_3', 'player_rating_away_player_4',
                   'player_rating_away_player_5', 'player_rating_away_player_6', 'player_rating_away_player_7',
                   'player_rating_away_player_8', 'player_rating_away_player_9',
                   'player_rating_away_player_10', 'player_rating_away_player_11', 'ewm_home_team_goals',
                   'ewm_away_team_goals', 'ewm_home_team_goals_conceded', 'ewm_away_team_goals_conceded',
                   'points_home', 'points_away', 'home_weighted_wins', 'away_weighted_wins', 'avg_home_team_rating',
                   'avg_away_team_rating', 'home_streak_wins', 'away_streak_wins', 'ewm_shoton_home',
                   'ewm_shoton_away', 'ewm_possession_home', 'ewm_possession_away', 'avg_home_rating_attack',
                   'avg_away_rating_attack', 'avg_away_rating_defence', 'avg_home_rating_defence',
                   'average_rating_home', 'average_rating_away', 'num_top_players_home', 'num_top_players_away',
                   'ewm_home_team_goals_conceded_x_ewm_shoton_home', 'attacking_strength_home',
                   'attacking_strength_away', 'attacking_strength_diff', 'result_match']

In [ ]:
import boto3
from sagemaker.session import Session
from sagemaker import clarify

default_prefix = "sagemaker/DEMO-sagemaker-clarify"
region = os.environ.get("AWS_REGION")
default_bucket = os.environ["BUCKET"]
execution_role = os.environ["ROLE"]
sagemaker_session = Session()

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=execution_role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    sagemaker_session=sagemaker_session
)

In [ ]:
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

train_uri = 's3://football-data-kamil/football-pipeline/2qrodexra55q/split-and-transform-data/output/train/train.csv'
train_input = TrainingInput(train_uri, content_type="text/csv")
test_uri = 's3://football-data-kamil/football-pipeline/2qrodexra55q/split-and-transform-data/output/test/test.csv'

In [ ]:
from io import StringIO
import pandas as pd

s3 = boto3.resource('s3')

obj = s3.Object(default_bucket, 'football-pipeline/2qrodexra55q/split-and-transform-data/output/train/train.csv')
data = obj.get()['Body']

In [ ]:
train_df = pd.read_csv(data, header=None)
train_df.columns = FEATURE_COLUMNS
# train_df

In [ ]:
bias_report_output_path = "s3://{}/{}/clarify-bias".format(default_bucket, default_prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_uri,
    s3_output_path=bias_report_output_path,
    label="result_match",
    headers=train_df.columns.to_list(),
    dataset_type="text/csv",
)

In [ ]:
MODEL_PACKAGE_GROUP = "football-group"

sm = boto3.client("sagemaker")
model_packages = sm.list_model_packages(ModelPackageGroupName=MODEL_PACKAGE_GROUP, SortBy="CreationTime", SortOrder="Descending")

model_package = [pk for pk in model_packages["ModelPackageSummaryList"] 
                 if pk["ModelApprovalStatus"] == "Approved"][0]

model_package_arn = model_package["ModelPackageArn"]

models = sm.search(
    Resource='Model',
    SearchExpression={
        'Filters': [
            {
                'Name': 'Model.Containers.ModelPackageName',
                'Operator': 'Equals',
                'Value': model_package_arn
            },
        ]
    }
)["Results"]

model_name = models[0]["Model"]["Model"]["ModelName"]

In [ ]:
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

In [ ]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="num_top_players_home", facet_values_or_threshold=[0], group_name="home_weighted_wins"
)

In [ ]:
from sagemaker.experiments import Run

with Run(
    experiment_name='tracking-bias-explainability',
    run_name="bias-only",
    sagemaker_session=sagemaker_session,
) as run:
    clarify_processor.run_bias(
        data_config=bias_data_config,
        bias_config=bias_config,
        model_config=model_config,
        model_predicted_label_config=predictions_config,
        pre_training_methods="all",
        post_training_methods="all",
    )